In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155620 - Copy.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155025_1 - Copy.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155427_1 - Copy.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_153313_1.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155351.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155611_1.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_162907_1 - Copy.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_153625_1.jpg
/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset/Cercospora/IMG_20230813_155613 -

In [2]:
# 1. Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
import random
import seaborn as sns
import cv2
from tqdm import tqdm
import copy
from PIL import Image
from sklearn.decomposition import PCA
from itertools import cycle
import os
import time

In [3]:

# Hyperparameters
DATA_PATH = "/kaggle/input/balanced-pomegranate-dataset/Pomegranate Diseases Dataset"
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.0002
IMG_SIZE = 224
NOISE_TYPE = 'speckle'
K_FOLDS = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Create output directory
os.makedirs("results", exist_ok=True)
os.makedirs("gradcam", exist_ok=True)


In [4]:

# 2. Enhanced Data Preprocessing with Noise Augmentation
class NoisyDataset(Dataset):
    def __init__(self, dataset, noise_type, noise_level=0.1):
        self.dataset = dataset
        self.noise_type = noise_type
        self.noise_level = noise_level

    def __len__(self):
        return len(self.dataset)

    def add_noise(self, img):
        img = img.clone()
        if self.noise_type == 'gaussian':
            noise = torch.randn(img.size()) * self.noise_level
            return torch.clamp(img + noise, 0, 1)
        elif self.noise_type == 'speckle':
            salt_pepper = torch.rand(img.size())
            img[salt_pepper < self.noise_level/2] = 0
            img[salt_pepper > 1 - self.noise_level/2] = 1
            return img
        elif self.noise_type == 'poisson':
            vals = len(torch.unique(img))
            vals = 2 ** np.ceil(np.log2(vals))
            noisy = torch.poisson(img * vals) / float(vals)
            return noisy
        return img

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return self.add_noise(img), label

# Preprocessing transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
base_dataset = ImageFolder(root=DATA_PATH, transform=transform)
class_names = base_dataset.classes
num_classes = len(class_names)
print(f"Found {len(base_dataset)} images in {num_classes} classes")


Found 5000 images in 5 classes


In [5]:

# 3. Enhanced Model with Grad-CAM++ Hooks
class FineTunedResNet101(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        base_model = models.resnet101(pretrained=True)
        
        # Feature extraction
        self.features = nn.Sequential(
            base_model.conv1,
            base_model.bn1,
            base_model.relu,
            base_model.maxpool,
            base_model.layer1,
            base_model.layer2,
            base_model.layer3,
            base_model.layer4
        )
        
        # Grad-CAM++ hooks
        self.activations = None
        self.gradients = None
        
        # Register hooks
        self.hook_handles = [
            self.features.register_forward_hook(self.forward_hook),
            self.features.register_full_backward_hook(self.backward_hook)
        ]
        
        # Classifier
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x).flatten(1)
        return self.classifier(x)
    
    def extract_features(self, x):
        with torch.no_grad():
            features = self.features(x)
            return self.adaptive_pool(features).flatten(1)
    
    def forward_hook(self, module, input, output):
        self.activations = output
    
    def backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
    
    def generate_cam(self):
        """Generate Grad-CAM++ heatmap"""
        if self.activations is None or self.gradients is None:
            return None
            
        with torch.no_grad():
            # Grad-CAM++ calculations
            gradients_squared = self.gradients.pow(2)
            gradients_cubed = self.gradients.pow(3)
            alpha = gradients_squared / (2 * gradients_squared + 
                                        gradients_cubed.sum(dim=(2, 3), keepdim=True) + 1e-6)
            weights = (alpha * torch.clamp(self.gradients, min=0)).sum(dim=(2, 3), keepdim=True)
            
            cam = (weights * self.activations).sum(dim=1, keepdim=True)
            cam = torch.relu(cam)
            cam_min = cam.min()
            cam_max = cam.max()
            if cam_max - cam_min > 0:
                cam = (cam - cam_min) / (cam_max - cam_min)
            else:
                cam = torch.zeros_like(cam)
        return cam
    
    def release_hooks(self):
        for handle in self.hook_handles:
            handle.remove()


In [6]:

# 4. Enhanced GSO Implementation
def fitness_function(x, features, labels):
    selected = x > 0.5  # Convert continuous to binary
    if np.sum(selected) == 0:
        return 0.0
    X = features[:, selected]
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, stratify=labels)
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    return knn.score(X_test, y_test)

def initialize_population(pop_size, dim, bounds):
    return np.random.uniform(bounds[0], bounds[1], (pop_size, dim)), np.zeros((pop_size, dim))

def tournament_selection(pop, fitness, tournament_size):
    selected = []
    for _ in range(len(pop)):
        candidates = np.random.choice(len(pop), tournament_size, replace=False)
        selected.append(pop[candidates[np.argmax(fitness[candidates])]])
    return np.array(selected)

def uniform_crossover(parent1, parent2, crossover_rate):
    if np.random.rand() < crossover_rate:
        mask = np.random.randint(0, 2, parent1.shape)
        return np.where(mask, parent1, parent2), np.where(mask, parent2, parent1)
    return parent1.copy(), parent2.copy()

def mutate(child, mutation_rate, bounds):
    mask = np.random.rand(len(child)) < mutation_rate
    child[mask] = np.random.uniform(bounds[0], bounds[1], sum(mask))
    return child

def gso_optimizer(features, labels, pop_size=30, dim=100, bounds=(0,1), hc=0.5,
                 max_iter=50, ga_params=(0.8, 0.1, 3), pso_params=(0.7, 1.5, 1.5)):
    # Initialize population
    pop, velocities = initialize_population(pop_size, dim, bounds)
    fitness = np.array([fitness_function(ind, features, labels) for ind in pop])

    pbest_positions = pop.copy()
    pbest_fitness = fitness.copy()
    gbest_idx = np.argmax(pbest_fitness)
    gbest_position = pbest_positions[gbest_idx].copy()
    gbest_fitness = pbest_fitness[gbest_idx]

    omega, phi1, phi2 = pso_params
    crossover_rate, mutation_rate, tournament_size = ga_params

    for iter in range(max_iter):
        # GA operations
        n_ga = int(hc * pop_size)
        ga_indices = np.random.choice(pop_size, n_ga, replace=False)
        selected = tournament_selection(pop[ga_indices], fitness[ga_indices], tournament_size)
        offspring = []
        for i in range(0, len(selected), 2):
            p1, p2 = selected[i], selected[i+1] if i+1 < len(selected) else selected[i]
            c1, c2 = uniform_crossover(p1, p2, crossover_rate)
            offspring.extend([mutate(c1, mutation_rate, bounds), mutate(c2, mutation_rate, bounds)])
        pop[ga_indices] = np.array(offspring[:n_ga])

        # PSO operations
        pso_indices = np.setdiff1d(np.arange(pop_size), ga_indices)
        for i in pso_indices:
            r1, r2 = np.random.rand(2)
            velocities[i] = omega*velocities[i] + phi1*r1*(pbest_positions[i]-pop[i]) + phi2*r2*(gbest_position-pop[i])
            pop[i] = np.clip(pop[i] + velocities[i], bounds[0], bounds[1])

        # Update fitness
        fitness = np.array([fitness_function(ind, features, labels) for ind in pop])
        improved = fitness > pbest_fitness
        pbest_positions[improved] = pop[improved]
        pbest_fitness[improved] = fitness[improved]

        if pbest_fitness.max() > gbest_fitness:
            gbest_idx = np.argmax(pbest_fitness)
            gbest_position = pbest_positions[gbest_idx].copy()
            gbest_fitness = pbest_fitness[gbest_idx]

        print(f"Iteration {iter+1}/{max_iter}: Best Fitness = {gbest_fitness:.4f}")

    return gbest_position > 0.5  # Return binary mask


In [7]:

# 5. Enhanced Training and Evaluation Functions
def train_model(model, train_loader, criterion, optimizer, scheduler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    
    if scheduler:
        scheduler.step()
    
    return epoch_loss, epoch_acc

def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs

def extract_features(model, loader):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc="Extracting features"):
            inputs = inputs.to(DEVICE)
            feats = model.extract_features(inputs).cpu().numpy()
            features.append(feats)
            labels.append(targets.numpy())
    return np.vstack(features), np.concatenate(labels)


In [8]:
# 6. Enhanced Visualization Functions
def visualize_gradcam(model, dataloader, fold, num_samples=5):
    model.eval()
    samples = random.sample(range(len(dataloader.dataset)), num_samples)
    
    plt.figure(figsize=(15, 3 * num_samples))
    
    for i, idx in enumerate(samples):
        image, true_label = dataloader.dataset[idx]
        input_tensor = image.unsqueeze(0).to(DEVICE)
        
        # Forward pass
        output = model(input_tensor)
        _, pred = torch.max(output, 1)
        pred_class = pred.item()
        
        # Generate CAM
        model.zero_grad()
        output[0, pred_class].backward(retain_graph=True)
        cam = model.generate_cam()
        
        if cam is None:
            continue
            
        # Process CAM
        cam = cam.squeeze().cpu().detach().numpy()
        cam = cv2.resize(cam, (IMG_SIZE, IMG_SIZE))
        cam = np.uint8(255 * cam)
        heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        
        # Original image
        denorm = transforms.Compose([
            transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
            transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
            transforms.ToPILImage()
        ])
        orig_img = denorm(image.cpu())
        orig_img = np.array(orig_img)
        
        # Overlay heatmap
        overlay = cv2.addWeighted(orig_img, 0.6, heatmap, 0.4, 0)
        
        # Plot
        plt.subplot(num_samples, 3, 3*i+1)
        plt.imshow(orig_img)
        plt.title(f"Original\nLabel: {class_names[true_label]}")
        plt.axis('off')
        
        plt.subplot(num_samples, 3, 3*i+2)
        plt.imshow(heatmap)
        plt.title(f"Grad-CAM++ Heatmap")
        plt.axis('off')
        
        plt.subplot(num_samples, 3, 3*i+3)
        plt.imshow(overlay)
        plt.title(f"Overlay\nPred: {class_names[pred_class]}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f"gradcam/fold_{fold+1}_gradcam.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Release hooks after visualization
    model.release_hooks()

def plot_roc_curve(y_true, y_score, class_names, fold):
    """Plot ROC curve with enhanced visualization"""
    # Binarize the labels
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Compute micro-average ROC curve and area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Plot ROC curves
    plt.figure(figsize=(10, 8))
    
    # Get color map
    colors = plt.cm.get_cmap('viridis', len(class_names))
    
    # Generate color values
    color_values = colors(np.linspace(0, 1, len(class_names)))
    
    # Plot each class
    for i, color in zip(range(len(class_names)), color_values):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')
    
    # Plot micro-average
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average (AUC = {roc_auc["micro"]:.2f})',
             color='deeppink', linestyle=':', linewidth=4)
    
    # Plot diagonal
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    
    # Format plot
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'results/fold_{fold+1}_roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return roc_auc

In [9]:

def plot_learning_curves(history_orig, history_noisy, fold):
    """Plot loss and accuracy curves for original and noisy models"""
    plt.figure(figsize=(15, 10))
    
    # Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(history_orig['train_loss'], label='Original Model', linestyle='-', linewidth=2)
    plt.plot(history_noisy['train_loss'], label='Noisy Model', linestyle='--', linewidth=2)
    plt.title('Training Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    plt.plot(history_orig['val_loss'], label='Original Model', linestyle='-', linewidth=2)
    plt.plot(history_noisy['val_loss'], label='Noisy Model', linestyle='--', linewidth=2)
    plt.title('Validation Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Accuracy curves
    plt.subplot(2, 2, 3)
    plt.plot(history_orig['train_acc'], label='Original Model', linestyle='-', linewidth=2)
    plt.plot(history_noisy['train_acc'], label='Noisy Model', linestyle='--', linewidth=2)
    plt.title('Training Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1.0)
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 2, 4)
    plt.plot(history_orig['val_acc'], label='Original Model', linestyle='-', linewidth=2)
    plt.plot(history_noisy['val_acc'], label='Noisy Model', linestyle='--', linewidth=2)
    plt.title('Validation Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1.0)
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'results/fold_{fold+1}_learning_curves.png', dpi=300)
    plt.close()

def plot_feature_pca(features, labels, title, fold):
    """Visualize features using PCA"""
    pca = PCA(n_components=2)
    components = pca.fit_transform(features)
    
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(components[:, 0], components[:, 1], c=labels, 
                         cmap='viridis', alpha=0.6, edgecolor='k', s=40)
    plt.colorbar(scatter, label='Class')
    plt.title(title)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(alpha=0.3)
    plt.savefig(f"results/fold_{fold+1}_{title.replace(' ', '_')}.png", dpi=300)
    plt.close()


In [10]:

# 7. K-Fold Cross Validation with Enhanced Tracking
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
results = {}
fold_accuracies = []
history = {}
gso_masks = []
mlp_classifiers = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(base_dataset)):
    fold_start = time.time()
    print(f"\n{'='*40}")
    print(f"STARTING FOLD {fold+1}/{K_FOLDS}")
    print(f"{'='*40}")
    
    # Create datasets
    train_base = Subset(base_dataset, train_ids)
    test_base = Subset(base_dataset, test_ids)
    
    # Apply noise augmentation
    train_noisy = NoisyDataset(train_base, NOISE_TYPE)
    test_noisy = NoisyDataset(test_base, NOISE_TYPE)
    
    # Create dataloaders
    train_loader_orig = DataLoader(train_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    test_loader_orig = DataLoader(test_base, batch_size=BATCH_SIZE, num_workers=4)
    train_loader_noisy = DataLoader(train_noisy, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    test_loader_noisy = DataLoader(test_noisy, batch_size=BATCH_SIZE, num_workers=4)
    
    # Initialize models
    model_orig = FineTunedResNet101(num_classes=num_classes).to(DEVICE)
    model_noisy = FineTunedResNet101(num_classes=num_classes).to(DEVICE)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer_orig = optim.Adam(model_orig.parameters(), lr=LEARNING_RATE)
    optimizer_noisy = optim.Adam(model_noisy.parameters(), lr=LEARNING_RATE)
    scheduler_orig = optim.lr_scheduler.StepLR(optimizer_orig, step_size=3, gamma=0.1)
    scheduler_noisy = optim.lr_scheduler.StepLR(optimizer_noisy, step_size=3, gamma=0.1)
    
    # Train both models with history tracking
    print("\nTraining original model:")
    orig_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_model(
            model_orig, train_loader_orig, criterion, optimizer_orig, scheduler_orig)
        val_loss, val_acc, _, _, _ = evaluate_model(model_orig, test_loader_orig, criterion)
        
        orig_history['train_loss'].append(train_loss)
        orig_history['train_acc'].append(train_acc)
        orig_history['val_loss'].append(val_loss)
        orig_history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")
    
    print("\nTraining noisy model:")
    noisy_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_model(
            model_noisy, train_loader_noisy, criterion, optimizer_noisy, scheduler_noisy)
        val_loss, val_acc, _, _, _ = evaluate_model(model_noisy, test_loader_noisy, criterion)
        
        noisy_history['train_loss'].append(train_loss)
        noisy_history['train_acc'].append(train_acc)
        noisy_history['val_loss'].append(val_loss)
        noisy_history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")
    
    # Store history
    history[fold] = {
        'orig': orig_history,
        'noisy': noisy_history
    }
    
    # Plot learning curves
    plot_learning_curves(orig_history, noisy_history, fold)
    
    # Feature extraction
    print("\nExtracting features...")
    features_orig, labels_orig = extract_features(model_orig, train_loader_orig)
    features_noisy, labels_noisy = extract_features(model_noisy, train_loader_noisy)
    
    # Feature fusion
    fused_features = np.concatenate((features_orig, features_noisy), axis=1)
    print(f"Fused features shape: {fused_features.shape}")
    
    # GSO feature selection
    print("\nRunning GSO optimization...")
    best_mask = gso_optimizer(fused_features, labels_orig, dim=fused_features.shape[1],
                             pop_size=30, max_iter=50, bounds=(0,1))
    selected_features = fused_features[:, best_mask]
    selected_count = np.sum(best_mask)
    print(f"Selected {selected_count} features ({selected_count/fused_features.shape[1]*100:.2f}%)")
    
    # Train MLP classifier on selected features
    X_train, X_val, y_train, y_val = train_test_split(
        selected_features, labels_orig, test_size=0.2, stratify=labels_orig)
    
    clf = MLPClassifier(
        hidden_layer_sizes=(512, 256),
        max_iter=1000,
        early_stopping=True,
        n_iter_no_change=20,
        random_state=42
    )
    clf.fit(X_train, y_train)
    val_acc = clf.score(X_val, y_val)
    print(f"MLP Validation Accuracy: {val_acc:.4f}")
    
    # Prepare test features
    test_features_orig, test_labels = extract_features(model_orig, test_loader_orig)
    test_features_noisy, _ = extract_features(model_noisy, test_loader_noisy)
    fused_test_features = np.concatenate((test_features_orig, test_features_noisy), axis=1)
    selected_test_features = fused_test_features[:, best_mask]
    
    # Final evaluation
    test_acc = clf.score(selected_test_features, test_labels)
    fold_accuracies.append(test_acc)
    print(f"\nTEST ACCURACY FOR FOLD {fold+1}: {test_acc:.4f}")
    
    # Save predictions for comprehensive report
    test_preds = clf.predict(selected_test_features)
    test_probs = clf.predict_proba(selected_test_features)
    
    # Save results
    results[fold] = {
        'test_labels': test_labels,
        'test_preds': test_preds,
        'test_probs': test_probs,
        'test_acc': test_acc,
        'gso_mask': best_mask,
        'selected_count': selected_count
    }
    
    gso_masks.append(best_mask)
    mlp_classifiers.append(clf)
    
    # Grad-CAM visualization
    print("\nGenerating Grad-CAM++ visualizations...")
    visualize_gradcam(model_noisy, test_loader_noisy, fold, num_samples=3)
    
    # Feature visualization
    print("\nVisualizing feature spaces...")
    plot_feature_pca(features_orig, labels_orig, "PCA of Original Features", fold)
    plot_feature_pca(features_noisy, labels_orig, "PCA of Noisy Features", fold)
    plot_feature_pca(fused_features, labels_orig, "PCA of Fused Features", fold)
    plot_feature_pca(selected_features, labels_orig, "PCA of Selected Features after GSO", fold)
    
    # ROC Curve
    print("\nGenerating ROC curve...")
    roc_auc_values = plot_roc_curve(test_labels, test_probs, class_names, fold)
    print(f"ROC AUC Values: {roc_auc_values}")
    
    # Confusion Matrix
    print("\nGenerating confusion matrix...")
    cm = confusion_matrix(test_labels, test_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - Fold {fold+1}')
    plt.savefig(f'results/fold_{fold+1}_confusion_matrix.png', dpi=300)
    plt.close()
    
    # Classification Report
    report = classification_report(test_labels, test_preds, target_names=class_names)
    print(f"\nClassification Report for Fold {fold+1}:\n{report}")
    
    fold_time = time.time() - fold_start
    print(f"\nFOLD {fold+1} COMPLETED IN {fold_time//60:.0f}m {fold_time%60:.0f}s")



STARTING FOLD 1/5


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 210MB/s]  



Training original model:


Evaluating: 100%|██████████| 32/32 [01:02<00:00,  1.94s/it]


Epoch 1/10 | Train Loss: 0.2020 | Acc: 0.9413 | Val Loss: 0.0776 | Acc: 0.9760


...

Fused features shape: (4000, 4096)

Running GSO optimization...
Iteration 1/50: Best Fitness = 1.0000
Iteration 2/50: Best Fitness = 1.0000
Iteration 3/50: Best Fitness = 1.0000
Iteration 4/50: Best Fitness = 1.0000
Iteration 5/50: Best Fitness = 1.0000
Iteration 6/50: Best Fitness = 1.0000
Iteration 7/50: Best Fitness = 1.0000
Iteration 8/50: Best Fitness = 1.0000
Iteration 9/50: Best Fitness = 1.0000
Iteration 10/50: Best Fitness = 1.0000
Iteration 11/50: Best Fitness = 1.0000
Iteration 12/50: Best Fitness = 1.0000
Iteration 13/50: Best Fitness = 1.0000
Iteration 14/50: Best Fitness = 1.0000
Iteration 15/50: Best Fitness = 1.0000
Iteration 16/50: Best Fitness = 1.0000
Iteration 17/50: Best Fitness = 1.0000
Iteration 18/50: Best Fitness = 1.0000
Iteration 19/50: Best Fitness = 1.0000
Iteration 20/50: Best Fitness = 1.0000
Iteration 21/50: Best Fitness = 1.0000
Iteration 22/50: Best Fitness = 1.0000
Iteration 23/50: Best Fitness = 1.0000
Iteration 24/50: Best Fitness = 1.0000
Iteration

Extracting features: 100%|██████████| 32/32 [00:43<00:00,  1.35s/it]



TEST ACCURACY FOR FOLD 5: 0.9910

Generating Grad-CAM++ visualizations...

Visualizing feature spaces...

Generating ROC curve...


/tmp/ipykernel_76/2624469537.py:88: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap('viridis', len(class_names))


ROC AUC Values: {0: 0.999895615185381, 1: 0.9999676415198131, 2: 0.9998150700049514, 3: 1.0, 4: 1.0, 'micro': 0.99995325}

Generating confusion matrix...

Classification Report for Fold 5:
                  precision    recall  f1-score   support

      Alternaria       0.98      0.98      0.98       189
     Anthracnose       0.99      0.99      0.99       191
Bacterial_Blight       0.99      0.98      0.98       213
      Cercospora       1.00      1.00      1.00       204
         Healthy       1.00      1.00      1.00       203

        accuracy                           0.99      1000
       macro avg       0.99      0.99      0.99      1000
    weighted avg       0.99      0.99      0.99      1000


FOLD 5 COMPLETED IN 87m 30s


In [12]:
import numpy as np
# 8. Final Performance Analysis
print("\n\nFINAL RESULTS:")
avg_acc = np.mean(fold_accuracies)
std_acc = np.std(fold_accuracies)
print(f"Average Test Accuracy: {avg_acc:.4f} ± {std_acc:.4f}")
print(f"Fold Accuracies: {[f'{acc:.4f}' for acc in fold_accuracies]}")

# Save final results
final_results = {
    'fold_accuracies': fold_accuracies,
    'avg_acc': avg_acc,
    'std_acc': std_acc,
    'class_names': class_names,
    'history': history,
    'gso_masks': gso_masks,
    'mlp_classifiers': mlp_classifiers
}
torch.save(final_results, 'results/final_results.pth')

# Generate comprehensive report
all_test_labels = np.concatenate([results[fold]['test_labels'] for fold in range(K_FOLDS)])
all_test_preds = np.concatenate([results[fold]['test_preds'] for fold in range(K_FOLDS)])

final_report = classification_report(all_test_labels, all_test_preds, target_names=class_names)
print("\nOverall Classification Report:")
print(final_report)

with open('results/classification_report.txt', 'w') as f:
    f.write(final_report)

# Plot feature selection statistics
selected_counts = [results[fold]['selected_count'] for fold in range(K_FOLDS)]
plt.figure(figsize=(10, 6))
plt.bar(range(1, K_FOLDS+1), selected_counts, color='skyblue')
plt.xlabel('Fold')
plt.ylabel('Number of Selected Features')
plt.title('Feature Selection by GSO Across Folds')
plt.xticks(range(1, K_FOLDS+1))
plt.grid(axis='y', alpha=0.3)
plt.savefig('results/feature_selection_stats.png', dpi=300)
plt.close()

print("\nTraining complete! All results saved.")



FINAL RESULTS:
Average Test Accuracy: 0.9906 ± 0.0016
Fold Accuracies: ['0.9930', '0.9900', '0.9910', '0.9880', '0.9910']

Overall Classification Report:
                  precision    recall  f1-score   support

      Alternaria       0.99      0.98      0.99      1000
     Anthracnose       0.99      0.99      0.99      1000
Bacterial_Blight       0.98      0.99      0.98      1000
      Cercospora       0.99      0.99      0.99      1000
         Healthy       1.00      1.00      1.00      1000

        accuracy                           0.99      5000
       macro avg       0.99      0.99      0.99      5000
    weighted avg       0.99      0.99      0.99      5000


Training complete! All results saved.
